# Cleaning the credit data

The code in the cell below was utilized from Activity 4 of Day 1 of Big Data week from our instructor demo

In [ ]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,763 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packa

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
# Start Spark session
from pyspark.sql import SparkSession
from pyspark import SparkFiles

The code in the cell below was utilized from Activity 4 of Day 1 of Big Data week from our instructor demo

In [ ]:
# Start Spark session
spark = SparkSession.builder.appName("DataFrame Basics").getOrCreate()

The code in the cell below was utilized from Activity 4 of Day 1 of Big Data week from our instructor demo




In [ ]:
# Read in data from our git repo
credit_url = 'https://raw.githubusercontent.com/cindyd97/Project_4_Group_4/main/Resources/Credit_data.csv'
spark.sparkContext.addFile(credit_url)
credit_df = spark.read.csv(SparkFiles.get("Credit_data.csv"), sep=",", header=True)

# Show DataFrame
credit_df.show()

+------+-----------+---------+---------------+---+-----------+------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+------------------+
|    ID|Customer_ID|    Month|           Name|Age|        SSN|  Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|
+------+-----------+---------+---------------+---+-----------+------------+---------

The code in the cell below was utilized from Activity 4 of Day 1 of Big Data week from our instructor demo




In [ ]:
# Convert spark dataframe to a pandas dataframe for cleaning purposes
credit_df = credit_df.toPandas()

In [ ]:
# Display the first 5 rows of data
credit_df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333328,3,...,2022.0,Good,809.98,35.03040185583525,22 Years and 9 Months,No,49.57494921489417,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.8433333333328,3,...,4.0,Good,809.98,33.053114497012245,22 Years and 10 Months,No,49.57494921489417,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.8433333333328,3,...,4.0,Good,809.98,33.811894118574465,NA,No,49.57494921489417,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,None,3,...,4.0,Good,809.98,32.430559017296765,23 Years and 0 Months,No,49.57494921489417,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986666666666,2,...,5.0,Good,605.03,25.92682170377555,27 Years and 3 Months,No,18.816214573128885,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [ ]:
# Check the data types for each column
credit_df.dtypes

,0
ID,object
Customer_ID,object
Month,object
Name,object
Age,object
SSN,object
Occupation,object
Annual_Income,object
Monthly_Inhand_Salary,object
Num_Bank_Accounts,object


In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
ID,0
Customer_ID,0
Month,0
Name,5015
Age,0
SSN,0
Occupation,0
Annual_Income,0
Monthly_Inhand_Salary,7498
Num_Bank_Accounts,0


In [ ]:
# Drop unnecessary columns (columns that are not needed for our features)
credit_df = credit_df.drop(['ID','Customer_ID','Name','SSN','Type_of_Loan'], axis=1)

In [ ]:
# Convert columns that are meant to be numeric to an integer
columns_to_convert = ['Age','Annual_Income','Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Num_Credit_Inquiries','Outstanding_Debt',
                      'Total_EMI_per_month','Amount_invested_monthly','Monthly_Balance','Monthly_Inhand_Salary','Num_Bank_Accounts','Num_Credit_Card','Interest_Rate',
                      'Delay_from_due_date','Credit_Utilization_Ratio']
credit_df[columns_to_convert] = credit_df[columns_to_convert].apply(pd.to_numeric, errors='coerce').astype('float64')

In [ ]:
# Check the data types for each column
credit_df.dtypes

,0
Month,object
Age,float64
Occupation,object
Annual_Income,float64
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,float64
Num_Credit_Card,float64
Interest_Rate,float64
Num_of_Loan,float64
Delay_from_due_date,float64


In [ ]:
# Check the unique values for each column in the dataframe (the group found random characters in the 'Payment_Behaviour' column)
credit_df['Payment_Behaviour'].value_counts()

,count
Payment_Behaviour,
Low_spent_Small_value_payments,12694
High_spent_Medium_value_payments,8922
High_spent_Large_value_payments,6844
Low_spent_Medium_value_payments,6837
High_spent_Small_value_payments,5651
Low_spent_Large_value_payments,5252
!@9#%8,3800


The code in the cell below was utilized from chatgpt

In [ ]:
# Remove random values in the 'Payment_Behaviour' column
credit_df = credit_df[~credit_df['Payment_Behaviour'].astype(str).str.contains(r'\!@9#%8', na=False)]

In [ ]:
# Check if the rows with random values have been removed from the dataframe for the 'Payment_Behaviour' column
credit_df['Payment_Behaviour'].value_counts()

,count
Payment_Behaviour,
Low_spent_Small_value_payments,12694
High_spent_Medium_value_payments,8922
High_spent_Large_value_payments,6844
Low_spent_Medium_value_payments,6837
High_spent_Small_value_payments,5651
Low_spent_Large_value_payments,5252


In [ ]:
# Check the unique values for each column in the dataframe (the group found rows with NA in the 'Credit_History_Age' column)
credit_df['Credit_History_Age'].value_counts()

,count
Credit_History_Age,
NA,4103
16 Years and 6 Months,240
16 Years and 1 Months,237
18 Years and 7 Months,235
20 Years and 1 Months,235
...,...
4 Years and 5 Months,19
0 Years and 11 Months,16
33 Years and 11 Months,14


The code in the cell below was utilized from chatgpt

In [ ]:
# Remove NA values in the 'Credit_History_Age' column
credit_df = credit_df[~credit_df['Credit_History_Age'].astype(str).str.contains(r'NA', na=False)]

In [ ]:
# check if the NA rows have been removed for the data frame for 'Credit_History_Age' column
credit_df['Credit_History_Age'].value_counts()

,count
Credit_History_Age,
16 Years and 6 Months,240
16 Years and 1 Months,237
18 Years and 7 Months,235
20 Years and 1 Months,235
19 Years and 7 Months,231
...,...
4 Years and 5 Months,19
0 Years and 11 Months,16
33 Years and 11 Months,14


The code in the cell below was utilized from chatgpt

In [ ]:
# Create a function that would convert the strings in 'Credit_History_Age' column to a float. (This column is a string that describes the amount of years and months
# the customer's credit age is)
def convert_to_float(years_months):
    if pd.isna(years_months):
        return 0.0
    parts = years_months.split()
    years = int(parts[0])
    months = int(parts[3])
    return round(years + (months / 12), 2)
# Apply function to convert
credit_df['Credit_History_Age'] = credit_df['Credit_History_Age'].apply(convert_to_float)

In [ ]:
# check if the 'Credit_History_Age' column was converted to a float for years and months
credit_df['Credit_History_Age'].value_counts()

,count
Credit_History_Age,
16.50,240
16.08,237
18.58,235
20.08,235
19.58,231
...,...
4.42,19
0.92,16
33.92,14


In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
Month,0
Age,2052
Occupation,0
Annual_Income,2968
Monthly_Inhand_Salary,6328
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,2045
Delay_from_due_date,0


In [ ]:
# Logically, the null values in the 'Age' column should not be changed to 0. Remove the rows with null values in this column
credit_df = credit_df.dropna(subset=['Age'])

In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
Month,0
Age,0
Occupation,0
Annual_Income,2822
Monthly_Inhand_Salary,6033
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,1943
Delay_from_due_date,0


The code in the cell below was utilized from chatgpt

In [ ]:
# The remaining columns with null values may be updated with the number 0
columns_to_fill = ['Annual_Income', 'Monthly_Inhand_Salary', 'Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Num_Credit_Inquiries','Outstanding_Debt',
                   'Amount_invested_monthly','Monthly_Balance']
credit_df[columns_to_fill] = credit_df[columns_to_fill].fillna(0)

In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
Month,0
Age,0
Occupation,0
Annual_Income,0
Monthly_Inhand_Salary,0
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,0
Delay_from_due_date,0


In [ ]:
# Check values in the 'Credit_Mix' column (label column)
credit_df['Credit_Mix'].value_counts()

,count
Credit_Mix,
Standard,14741
Good,9760
_,7884
Bad,7660


The code in the cell below was utilized from chatgpt

In [ ]:
# Remove null values in the 'Credit_Mix' column (label column)
credit_df = credit_df[~credit_df['Credit_Mix'].astype(str).str.contains(r'\_', na=False)]

The code in the cell below was utilized from Xpert learning asssistant

In [ ]:
# Assign each value with an integer
credit_df['Credit_Mix'] = credit_df['Credit_Mix'].replace({'Standard': 0, 'Good': 1, 'Bad': 2}).astype(int)

<ipython-input-27-4920fe07fc10>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  credit_df['Credit_Mix'] = credit_df['Credit_Mix'].replace({'Standard': 0, 'Good': 1, 'Bad': 2}).astype(int)


In [ ]:
# Check values in the 'Credit_Mix' column (label column)
credit_df['Credit_Mix'].value_counts()

,count
Credit_Mix,
0,14741
1,9760
2,7660


In [ ]:
# Download the cleaned data set
credit_df.to_csv('clean_credit_data.csv', index=False)
from google.colab import files
files.download("clean_credit_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cleaning the loan data

The code in the cell below was utilized from Activity 4 of Day 1 of Big Data week from our instructor demo

In [ ]:
# Read in data from our git repo
loan_url = 'https://raw.githubusercontent.com/cindyd97/Project_4_Group_4/main/Resources/Loan_default.csv'
spark.sparkContext.addFile(loan_url)
loan_df = spark.read.csv(SparkFiles.get("Loan_default.csv"), sep=",", header=True)

# Show DataFrame
loan_df.show()

+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|    LoanID|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|
+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|I38PQUQS96| 56| 85994|     50587|        520|            80|             4|       15.23|      36|    0.44| Bachelor's|     Full-time|     Divorced|        Yes|          Yes|      Other|        Yes|      0|
|HPSK72WA7R| 69| 50432|    124440|        458|            15|             1|        4.81|      60|    0.68|   Master's|     Full-time|      Married|         No|           N

The code in the cell below was utilized from Activity 4 of Day 1 of Big Data week from our instructor demo

In [ ]:
# Convert spark dataframe to a pandas dataframe for cleaning purposes
loan_df = loan_df.toPandas()

In [ ]:
# display first 5 rows of the data frame
loan_df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [ ]:
# Check the data types for each column
loan_df.dtypes

,0
LoanID,object
Age,object
Income,object
LoanAmount,object
CreditScore,object
MonthsEmployed,object
NumCreditLines,object
InterestRate,object
LoanTerm,object
DTIRatio,object


In [ ]:
# Check the null values in the data set
loan_df.isnull().sum()

,0
LoanID,0
Age,0
Income,0
LoanAmount,0
CreditScore,0
MonthsEmployed,0
NumCreditLines,0
InterestRate,0
LoanTerm,0
DTIRatio,0


In [ ]:
# Degault column will be our labeled values. Default (1), not Default (0)
loan_df['Default'].value_counts()

,count
Default,
0,225694
1,29653


In [ ]:
# Check for duplicate values
duplicate_rows = loan_df[loan_df.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [LoanID, Age, Income, LoanAmount, CreditScore, MonthsEmployed, NumCreditLines, InterestRate, LoanTerm, DTIRatio, Education, EmploymentType, MaritalStatus, HasMortgage, HasDependents, LoanPurpose, HasCoSigner, Default]
Index: []


In [ ]:
# Drop unnecessary columns (columns that are not needed for our features)
loan_df = loan_df.drop(['LoanID'], axis=1)

In [ ]:
loan_df.head()

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [ ]:
# Check the data types for each column
loan_df.dtypes

,0
Age,object
Income,object
LoanAmount,object
CreditScore,object
MonthsEmployed,object
NumCreditLines,object
InterestRate,object
LoanTerm,object
DTIRatio,object
Education,object


In [ ]:
# Convert columns that are meant to be numeric to an integer
columns_to_convert2 = ['Age','Income','LoanAmount','CreditScore','MonthsEmployed','NumCreditLines','InterestRate','LoanTerm','DTIRatio','Default']
loan_df[columns_to_convert2] = loan_df[columns_to_convert2].apply(pd.to_numeric, errors='coerce').astype('float64')

In [ ]:
# Check the data types for each column
loan_df.dtypes

,0
Age,float64
Income,float64
LoanAmount,float64
CreditScore,float64
MonthsEmployed,float64
NumCreditLines,float64
InterestRate,float64
LoanTerm,float64
DTIRatio,float64
Education,object


In [ ]:
# Download the cleaned data set
loan_df.to_csv('clean_loan_data.csv', index=False)
from google.colab import files
files.download("clean_loan_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>